# Journalistieke Vrijheid in Beeld: Invloed van Politieke Regimes en Conflicten
Groep: *D1 - Walvisualisatie*

Anne Beks - 15237621, 

Glenn de Graaff - 15149013, 

Noor Koevoet - 15054039, 

Job Kolhorn - 15217140


## Introductie
Persvrijheid is een principe dat stelt dat de pers vrij is om te schrijven, zeggen en publiceren wat zij willen. De overheid is in beginsel verboden om censuur toe te passen en mag niet vooraf controleren wat wel of niet gezegd mag worden. In Nederland is persvrijheid, samen met het recht op meningsuiting, vastgelegd in artikel 7 van de Grondwet (Ministerie van Justitie en Veiligheid, 2022). Ook in andere landen en binnen Europees verband zijn er wetten die persvrijheid en vrijheid van meningsuiting beschermen.

Om te verzekeren dat journalisten hun werk veilig kunnen doen, hebben zij een beschermde status van het Europese Hof voor de Rechten van de Mens (EHRM). Journalisten brengen informatie naar buiten over algemeen belangrijke zaken en zijn daarom van cruciaal belang voor de democratie. Hoewel er vrijheid is om te publiceren wat men wil, betekent dit niet dat dit onbestraft kan gebeuren. Wanneer publicaties discriminerend zijn of aanzetten tot opruiing, kan iemand aangeklaagd en veroordeeld worden. Dit gebeurt echter pas achteraf; vóór publicatie is censuur niet toegestaan.

In veel landen is persvrijheid echter niet gegarandeerd. In landen met een autocratisch regime is de vrijheid van meningsuiting vaak beperkt of volledig afwezig. Ook in landen waar conflicten gaande zijn, zijn er mogelijk meerdere machten aan de leiding, of lijkt er niemand aan de macht te zijn. In zulke situaties is het moeilijk om te waarborgen dat de wet wordt nageleefd.

Deze data-analyse zal onderzoeken of de mate van persvrijheid wordt beïnvloed door conflicten of de mate van democratie. Door gebruik te maken van data van Reporters Without Borders (RSF), The Armed Conflict Location & Event Data Project (ACLED) en Our World in Data, zullen we scores en ranglijsten analyseren rondom persvrijheid, democratie en conflicten wereldwijd.

Er zijn verschillende factoren die de mate van persvrijheid in een land kunnen beïnvloeden. Een belangrijk aspect is de aanwezigheid van conflicten. Conflicten kunnen de veiligheid van journalisten bedreigen, leiden tot verhoogde censuur en beperkingen opleggen aan de informatievoorziening. Hierbij zijn we bij het eerste perspectief gekomen: conflicten verminderen de persvrijheid. Daarnaast speelt het type bestuursvorm een cruciale rol. Autocratische regimes hebben vaak strenge controlemechanismen om kritiek en onafhankelijke berichtgeving te onderdrukken, waardoor de persvrijheid ernstig kan worden beperkt. Hieruit volgt het tweede perspectief: autocratische bestuursvormen verminderen de persvrijheid.


## Dataset en benodigde modules importeren

In [266]:
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.express as px

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np

from IPython.display import display

init_notebook_mode(connected=True)

In [ ]:
# Laad de dataset
eind_df = pd.read_pickle('./data/dataset.pkl')

## Conflicten Verminderen de Persvrijheid

### Argument 1

**Landen die in conflict zijn hebben op dat moment een lage persvrijheid.**

Het is algemeen bekend dat conflicten een negatieve impact hebben op verschillende aspecten van de samenleving, en persvrijheid vormt hierop geen uitzondering. Tijdens conflicten wordt de veiligheid van journalisten vaak in gevaar gebracht, waardoor ze hun werk niet vrij kunnen uitvoeren. De dreiging van geweld en intimidatie zorgt ervoor dat journalisten zichzelf censureren of stoppen met rapporteren over gevoelige onderwerpen. Dit heeft verstrekkende gevolgen voor de informatievoorziening en de kwaliteit van het nieuws dat het publiek bereikt. In conflictgebieden worden vaak strikte controlemaatregelen en censuur opgelegd om de verspreiding van informatie te beheersen, en worden journalisten vaak geconfronteerd met arrestaties, aanvallen en zelfs moord, wat leidt tot een omgeving waarin onafhankelijke berichtgeving vrijwel onmogelijk is.

De onderstaande pie chart (figuur 1) biedt een helder inzicht in de verdeling van conflictniveaus wereldwijd. De verdeling is weergegeven in vier kleuren voor de conflictstatus: blauw staat voor laag/inactief, oranje voor hoog, geel voor turbulent en rood voor extreem. Uit het diagram blijkt dat 71,4% van de landen momenteel een lage of inactieve conflictstatus heeft. In tegenstelling hiermee bevindt de overige 28,6% van de landen zich in een hogere conflictstatus. Dit kan betekenen dat bijna een derde van de journalisten in deze landen te maken heeft met aanzienlijke uitdagingen en belemmeringen bij hun werk door de situatie in hun land.


In [ ]:
# Creëer een pie chart van de verschillende conflict statussen

color_counts = eind_df['conflict_categorie_2023'].value_counts()

color_discrete_map = {
    "Low/Inactive": "blue",
    "High": "orange",
    "Turbulent": "yellow",
    "Extreme": "red",
}

fig = px.pie(eind_df, 
             values = color_counts.values,
             names = [color for color in color_counts.index],
             color=[color for color in color_counts.index], 
             color_discrete_map=color_discrete_map)


fig.update_traces(
    textposition='inside', 
    textinfo='percent',
    hovertemplate='<b>%{label}</b><br>Aantal landen: %{value}<extra></extra>')

fig.update_layout(
    title = 'Aantal landen met bepaalde conflict status (index level) ',
    legend_title = 'Index level'
)

fig.show()

print('Figuur 1')

De scatterplot hieronder (figuur 2) toont een significante negatieve correlatie tussen de conflict rangschikking en de persvrijheid rangschikking voor het jaar 2023. De visualisatie laat een correlatie van -0.43 zien tussen deze twee indexen. Dit betekent dat landen met een lage of inactieve conflictstatus (blauw) doorgaans lager scoren op de persvrijheid rangschikking, wat aangeeft dat er veel persvrijheid is. De scatterplot laat ook zien dat de 28,6% van de landen die zich in een turbulente, hoge of extreme conflictstatus bevinden, een hoge score hebben op de persvrijheid rangschikking, wat betekent dat er weinig persvrijheid is in deze landen.

In [282]:
# Maak een scatterplot van de conflict en persvrijheid rangen

# Definieer de kleuren voor elke categorie
color_discrete_map = {
    "Low/Inactive": "blue",
    "High": "orange",
    "Turbulent": "yellow",
    "Extreme": "red"
}

# Maak de scatterplot
fig = px.scatter(
    eind_df,
    x="conflict_rang_2023",  # Pas dit aan naar de gewenste x-as
    y="persvrijheid_rang_2023",  # Pas dit aan naar de gewenste y-as
    color="conflict_categorie_2023",
#     size=10,
    color_discrete_map=color_discrete_map,
    hover_name="land",
    title="Scatterplot van Conflict Rangschikking vs Persvrijheid Rangschikking"
)

fig.update_layout(legend_title_text='Intensiteit van conflicten')


# Update de opmaak
fig.update_layout(
    xaxis_title="Conflict Score 2023",
    yaxis_title="Persvrijheid Score 2023"
)

fig.show()

print('Figuur 2')

correlation = eind_df["conflict_rang_2023"].corr(eind_df["persvrijheid_rang_2023"])
print(f"Correlatie tussen de conflict rangschikking en de persvrijheid rangschikking: {correlation}")

Correlatie tussen de conflict index ranking en de persvrijheid index ranking: -0.42573893484944786


Samenvattend kunnen we stellen dat de aanwezigheid van conflicten een relatief sterk verband heeft met de vermindering van persvrijheid. Conflicten creëren een vijandige en gevaarlijke omgeving voor journalisten, wat leidt tot zelfcensuur, verhoogde censuur door de autoriteiten en uiteindelijk een vermindering van de beschikbaarheid van vrije, onafhankelijke en dus betrouwbare informatie. In een gezonde democratie is het cruciaal dat burgers toegang hebben tot onbevooroordeelde informatie om weloverwogen beslissingen te kunnen nemen en om machthebbers ter verantwoording te roepen. De gegevens en visualisaties ondersteunen het perspectief dat conflicten de persvrijheid verminderen.

### Argument 2

**Naarmate een conflict escaleert, verslechtert de persvrijheid nog meer.**

Naarmate conflicten escaleren, wordt de persvrijheid steeds meer onder druk gezet. Hoe meer het conflict escaleert, hoe groter het risico voor journalisten, wier leven steeds meer wordt bedreigd. Deze toenemende dreiging leidt tot zelfcensuur en het vermijden van gevoelige onderwerpen. Autoriteiten kunnen de escalerende situatie benutten om striktere censuur op te leggen, waardoor vrije en objectieve berichtgeving verder wordt beperkt. Deze escalatie maakt de situatie voor de persvrijheid erger en ondermijnt democratische processen door de toegang tot betrouwbare informatie te beperken.

De wereldkaart in figuur 3 illustreert de veranderingen in de rangorde van conflicten tussen 2019 en 2023 en de verandering in de rangorde van persvrijheid rang tussen 2019 en 2023. Groen betekent dat zowel persvrijheid als conflict zijn verbeterd, rood dat beide verslechterd zijn, oranje dat persvrijheid is verbeterd terwijl het conflict is verminderd, en geel dat persvrijheid is verslechterd terwijl het conflict is verbeterd. Deze kaart biedt een samenvattend overzicht van de dynamiek tussen conflict en persvrijheid wereldwijd. Enkele voorbeelden van landen waar de conflictrang is gedaald zijn Palestina, Israël en Papoea-Nieuw-Guinea. Israël is bijvoorbeeld 58 rangen gedaald, Palestina 50 en Papoea-Nieuw-Guinea 113. Ondanks dat Papoea-Nieuw-Guinea meer is gedaald in conflictrang, heeft Israël bijvoorbeeld toch een lagere rang op de conflict index ranking (wat betekent dat Israël dus meer conflict heeft). Als we kijken naar dezelfde drie landen, zien we dat de persvrijheid van Israël met 9 is gedaald, Palestina met 19 en Papoea-Nieuw-Guinea met 21. Ook hier is belangrijk om op te merken dat een grotere daling niet noodzakelijk betekent dat dit land het land is met de minste persvrijheid.


In [ ]:
# Creëer een wereldkaart om de verschillen tussen de veranderingen van persvrijheid en conflicten te laten zien

# Definieer kleuren op basis van categorieën
def get_color(verandering_pers, verandering_conf):
    if verandering_pers == 0 and verandering_conf == 0:
        return 'Pers en Conf neutraal'  # Blauw: beide veranderingen zijn 0
    elif (verandering_pers >= 0 and verandering_conf > 0) or (verandering_pers > 0 and verandering_conf >= 0):
        return 'Pers en Conf positief'  # Groen: beide veranderingen zijn positief
    elif verandering_pers < 0 and verandering_conf < 0:
        return 'Pers en Conf negatief'  # Rood: beide veranderingen zijn negatief
    elif verandering_pers < 0 and verandering_conf >= 0:
        return 'Pers negatief Conf positief'  # Geel: persvrijheid is negatief, conflict is positief
    elif verandering_pers >= 0 and verandering_conf < 0:
        return 'Pers positief Conf negatief'  # Oranje: persvrijheid is positief, conflict is negatief

# Voeg kleur toe aan DataFrame
eind_df['color'] = eind_df.apply(lambda row: get_color(row['persvrijheid_verandering'], row['conflict_verandering']), axis=1)

# Maak een lijst met kleuren op basis van categorie
color_discrete_map = {
    'Pers en Conf neutraal': 'blue',    
    'Pers en Conf positief': '#90ee90',
    'Pers en Conf negatief': 'red',     
    'Pers negatief Conf positief': 'yellow',
    'Pers positief Conf negatief': 'orange'
}

# Maak de choropletenkaart met Plotly Express
fig = px.choropleth(
    eind_df,
    locations="land",
    locationmode="country names",
    color="color",
    hover_name="land",
    hover_data={
        "color": False,
        "persvrijheid_verandering": True,
        "conflict_verandering": True
    },
    color_discrete_map=color_discrete_map,
    title="Verandering in Persvrijheid en Conflict Rang (2019-2023)"
)

# Aanpassen van de opmaak van de figuur
fig.update_geos(
    showocean=True,
    oceancolor='#a8d5f2',
    showcountries=True,
    countrycolor='white',
    landcolor='white',
    showland=True,
    projection_type='mercator'
)

fig.update_layout(
    width=900,
    height=600,
    margin={'t': 80, 'r': 80, 'b': 50, 'l': 80},
    title={
        'text': "Verandering in Persvrijheid en Conflict Rang (2019-2023)",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)

# Toon de figuur
fig.show()

print('Figuur 3')

In figuur 3 is duidelijk te zien dat de wereldkaart voornamelijk verdeeld is in rood en oranje. Deze kleuren kunnen echter een vertekend beeld geven, zoals blijkt uit specifieke voorbeelden. Er zijn weinig landen die tussen 2019 en 2023 zowel een verbetering hebben meegemaakt in de conflictrang als in de persvrijheidsrang (groen). Ook zijn er weinig landen te vinden waarbij de conflictrang is gestegen en de persvrijheid is gedaald (geel). Australië en Mongolië zijn beide geel, maar hebben geen verandering in hun conflictstatus meegemaakt, alleen in persvrijheid. 

Een opvallend voorbeeld is Amerika, waar de conflict rangschikking met 112 punten is gedaald, terwijl de persvrijheid rangschikking slechts met 3 punten is verbeterd. Frankrijk toont een vergelijkbare trend met een afname van 102 punten op de  conflict rangschikking en een verbetering van 8 punten de persvrijheid rangschikking. In contrast hiermee zijn Israël, Palestina en Papoea-Nieuw-Guinea allemaal roodgekleurd, wat duidt op een aanzienlijke verslechtering van zowel de conflicten als de persvrijheid in deze gebieden.

Deze gegevens versterken het argument dat naarmate het conflict escaleert, de persvrijheid nog meer verslechtert. Ondanks mogelijke verbeteringen in persvrijheid in sommige landen met een verslechterd conflict (zoals in Amerika en Frankrijk), blijft de algemene trend aangeven dat escalatie van conflicten vaak gepaard gaat met beperkingen op de journalistieke vrijheid.


## Autocratische Bestuursvormen Verminderen de Persvrijheid

### Argument 1

**Landen met een meer autocratische bestuursvorm hebben een lagere persvrijheid.**

Het verschil tussen democratische en autocratische bestuursvormen benadrukken is essentieel voor het begrijpen van de impact op persvrijheid. Democratische systemen kenmerken zich door politieke pluraliteit, burgerlijke vrijheden en transparantie, waarbij de pers een cruciale rol speelt in controle en openbaar debat. Autocratische regimes daarentegen concentreren de macht, beperken politieke vrijheden en onderdrukken oppositie en vrije media. Deze regimes gebruiken censuur, arrestaties van journalisten en controle over mediakanalen om kritiek te onderdrukken en de informatiestroom en publieke opinie te manipuleren.

In figuur 4 wordt een pie chart gepresenteerd die de verandering in de democratie rangschikking tussen 2019 en 2023 weergeeft. Hieruit blijkt dat 51.1% van de landen (groen) een verbetering in rang heeft ervaren, wat betekent dat ze democratischer zijn geworden. Daarentegen is 10.1% van de landen (blauw) gelijk gebleven, terwijl 38.8% van de landen (rood) een verslechtering heeft ondergaan, wat wijst op toenemende autocratische ontwikkelingen.

In [ ]:
# Creëer een pie chart van de verandering van de democratie rangen

status_counts = pd.Series(['Verslechterd' if x < 0 else 'Verbeterd' if x > 0 else 'Gelijk gebleven' for x in eind_df['democratie_verandering']]).value_counts()


fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['green', 'red', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Democratie rangschikking sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

print('Figuur 4')

Figuur 5 toont een wereldkaart die de gecombineerde veranderingen in de democratie rang en persvrijheid rang tussen 2019 en 2023 weergeeft. De kaart maakt gebruik van verschillende kleuren om deze veranderingen te illustreren. Oranje gebieden geven aan waar de persvrijheid is verbeterd, terwijl de democratie rang is verslechterd. Gele gebieden duiden op een verslechtering van de persvrijheid en een verbetering van de democratie rang. Rode gebieden wijzen op een verslechtering van zowel persvrijheid als democratie. Groene gebieden geven een verbetering aan zowel persvrijheid als democratie. Blauwe gebieden zijn neutraal gebleven, zonder significante veranderingen in persvrijheid of democratie.

In [ ]:
# Creëer een wereldkaart om de verschillen tussen de veranderingen van persvrijheid en democratie te laten zien

# Definieer kleuren op basis van categorieën
def get_color(verandering_pers, verandering_dem):
    if verandering_pers == 0 and verandering_dem == 0:
        return 0  # Blauw: beide veranderingen zijn 0
    elif (verandering_pers >= 0 and verandering_dem > 0) or (verandering_pers > 0 and verandering_dem >= 0):
        return 1  # Groen: beide veranderingen zijn positief
    elif verandering_pers < 0 and verandering_dem < 0:
        return 2  # Rood: beide veranderingen zijn negatief
    elif verandering_pers < 0 and verandering_dem >= 0:
        return 3  # Geel: persvrijheid is negatief, democratie is positief
    elif verandering_pers >= 0 and verandering_dem < 0:
        return 4  # Oranje: persvrijheid is positief, democratie is negatief

# Voeg kleur toe aan DataFrame
eind_df['color'] = eind_df.apply(lambda row: get_color(row['persvrijheid_verandering'], row['democratie_verandering']), axis=1)
 
# Maak een lijst met kleuren op basis van categorie
colorscale = [
    'blue',    # Blauw: beide veranderingen zijn 0
    '#90ee90', # Groen: beide veranderingen zijn positief
    'red',     # Rood: beide veranderingen zijn negatief
    'yellow',  # Geel: persvrijheid is negatief, democratie is positief
    'orange',  # Oranje: persvrijheid is positief, democratie is negatief
]
 
# Creëer de choropletenkaart
data = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['color'],  # Gebruik de 'color' kolom als numerieke waarde voor de kleuren
    hoverinfo='location+text',  # Toon locatie, kleurwaarde en tekst bij hover    
    text=eind_df.apply(lambda row: f'Persvrijheid: {row["persvrijheid_verandering"]}<br>Democratie: {row["democratie_verandering"]}', axis=1),
    marker_line_color='gray',  # Voeg een rand toe aan de markeringen
    colorscale=colorscale,
    showscale=False,  # Zet showscale uit om geen kleurenschaal te tonen
)
 
# Creëer de opmaak
layout = go.Layout(
    width=900,
    height=600,
    title='Verandering in Persvrijheid en Democratische Rang (2019-2023)',
    geo=dict(
        showocean=True,
        oceancolor='#a8d5f2',
        showcountries=True,
        countrycolor='white',
        landcolor='white',
        showland=True,
        projection_type='mercator'
    ),
    margin={'t': 50, 'r': 90, 'b': 50, 'l': 40},
)
 
# Voeg de legenda handmatig toe als annotaties
legend_items = {
    'Pers en Demo neutraal': 0,
    'Pers en Demo positief': 1,
    'Pers en Demo negatief': 2,
    'Pers negatief Demo positief': 3,
    'Pers positief Demo negatief': 4,
}
 
annotations = []
for label, color_index in legend_items.items():
    annotations.append(dict(
        x=0.93,
        y=0.95 - (len(colorscale) - color_index - 1) * 0.05,
        xref='paper',
        yref='paper',
        text=label,
        showarrow=False,
        font=dict(size=10),
        align='left',
        xanchor='left',
    ))

# Voeg de legenda bolletjes toe
for color_index, color_value in enumerate(colorscale):
    annotations.append(dict(
        x=0.92,
        y=0.95 - (len(colorscale) - color_index - 1) * 0.05,
        xref='paper',
        yref='paper',
        text = '',
        showarrow=False,
        bgcolor=colorscale[color_index],
        width=13,
        height=13,
        borderpad=0,
    ))

layout['annotations'] = annotations
 
# Maak de figuur
fig = go.Figure(data=[data], layout=layout)
 
# Toon de figuur
fig.show()

print('Figuur 5')

In figuur 5 zijn veel kleuren te zien, maar we focussen ons vooral op de rode en oranje landen. Deze landen zijn van 2019 tot 2023 meer geneigd naar een autocratische bestuursvorm. Hoewel sommige van deze landen slechts een lage verschuiving hebben van -15 of minder in hun democratie rang, lijkt dit misschien niet significant. Het is echter logisch dat een land niet zomaar van democratisch naar autocratisch verschuift, waardoor een verplaatsing van bijvoorbeeld 100 rangen heel onwaarschijnlijk is.

De rode landen hebben een duidelijke daling in persvrijheid meegemaakt, terwijl de oranje landen over het algemeen slechts kleine stijgingen in persvrijheid hebben laten zien. Deze wereldkaart toont een subtiele relatie tussen de afname van democratische normen en de verslechtering van persvrijheid, waarbij opvalt dat sommige autocratische landen (oranje) ondanks een meer autocratisch bestuur toch een kleine verbetering in persvrijheid laten zien.

Figuur 6 is een scatterplot die de relatie toont tussen de democratie rangschikking 2023 en de persvrijheid rangschikking 2023. Deze grafiek biedt een visuele representatie van de dynamiek tussen bestuursvorm en persvrijheid. Een lagere score op de democratie rangschikking duidt op een meer democratische bestuursvorm, terwijl een hogere score wijst op een meer autocratische bestuursvorm. Voor de persvrijheids rangschikking geldt dat een lage score gelijk staat aan meer persvrijheid, terwijl een hoge score minder persvrijheid betekent.

In [284]:
# creeer een scatterplot om de relatie tussen de democratie en persvrijheid aan te geven per land

fig = px.scatter(
    eind_df,
    x='democratie_rang_2023',
    y='persvrijheid_rang_2023',
    title = 'Democracy vs Persvrijheid rangschikking 2023 ',
    hover_data=['land'],
    trendline="ols")

fig.update_layout(
    xaxis_title="Democratie rangschikking 2023",
    yaxis_title="Persvrijheid rangschikking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"

fig.show()

print('Figuur 6')

correlation = eind_df["democratie_rang_2023"].corr(eind_df["persvrijheid_rang_2023"])
print(f"Correlatie tussen de democratie rangschikking en de persvrijheid rangschikking: {correlation}")

In figuur 6 te zien dat naarmate de democratische normen afnemen en een land autocratischer wordt, de persvrijheids rangschikking stijgt, wat neerkomt op minder persvrijheid. Deze scatterplot toont een sterke correlatie van ongeveer 0.75, wat een duidelijk verband aantoont tussen de bestuursvorm en de persvrijheid.

### Argument 2

**Landen met een meer autocratische bestuursvorm hebben vaker conflicten.**

Autocratische bestuursvormen concentreren de macht in handen van een enkele leider of een kleine groep, met beperkte politieke vrijheden en burgerrechten. In deze systemen worden dissidente stemmen vaak onderdrukt en media gecontroleerd. Intern kunnen autocratische regimes geweld en repressie gebruiken om oppositie te onderdrukken, wat kan leiden tot opstanden en burgeroorlogen. Daarnaast kunnen ze militaire agressie inzetten om hun macht te versterken en territoriale ambities na te streven. Deze focus op controle en macht maakt autocratische regimes vaak geneigd om conflicten te escaleren, wat zowel nationaal als internationaal verstrekkende gevolgen heeft. Begrip van deze dynamiek is essentieel voor het bevorderen van vrede en stabiliteit.

Figuur 7 toont een scatterplot van de democratie rangschikking versus de conflict rangschikking in 2023. In deze grafiek zien we een lichte correlatie tussen de mate van democratie en het aantal conflicten in een land. Een hogere democratie rang betekent een meer autocratische bestuursvorm. Bij de conflict rangschikking betekent een hogere rang minder conflicten en een lagere rang dus meer conflicten.


In [ ]:
# creeer een scatterplot om de relatie tussen de democratie en conflict aan te geven per land

fig = px.scatter(
    eind_df,
    x='democratie_rang_2023',
    y='conflict_rang_2023',
    title = 'Democratie rangschikking vs Conflict rangschikking 2023',
    hover_data=['land'],
    trendline="ols",
)

fig.update_layout(
    xaxis_title="Democratie rangschikking 2023",
    yaxis_title="Conflict rangschikking 2023"
)

for trace in fig.data:
    if trace.mode == "lines":
        trace.line.color = "black"


fig.show()

print('Figuur 7')

correlation = eind_df["democratie_rang_2023"].corr(eind_df["conflict_rang_2023"])
print(f"Correlatie tussen de democratie rangschikking en de conflict rangschikking: {correlation}")

De scatterplot laat zien dat autocratische landen zich vaak lager op de conflict rangschikking bevinden, wat betekent dat deze landen doorgaans meer conflicten ervaren. Dit patroon suggereert dat autocratische regimes mogelijk een grotere neiging hebben tot interne en externe conflicten, waarschijnlijk door hun repressieve en ondoorzichtige bestuur.

Hoewel de correlatie niet extreem sterk is, biedt deze scatterplot toch waardevolle inzichten in hoe de bestuursvorm van een land kan samenhangen met zijn neiging tot conflicten. Autocratische regimes lijken vaker betrokken te zijn bij conflicten, wat mogelijk te wijten is aan factoren zoals gebrek aan politieke participatie, onderdrukking van oppositie en beperkingen op burgerlijke vrijheden.

## Reflectie

Op donderdag 20 juni hebben wij tijdens het college onze draft gepresenteerd. Daarna ontvingen we zowel feedback van onze TA als peer feedback van groep B1.Een belangrijk punt dat zowel door de tutor als het andere groepje werd opgemerkt, was dat de ranking van de verschillende indexen onduidelijk was. In de conflictindex stond een index van 1 voor veel conflicten, wat negatief is, terwijl in de democratie- en persvrijheidsindex een ranking van 1 juist positief was.

Een aantal visualisaties werkten niet in Jupyter Book omdat het een statische omgeving is en de visualisaties dynamisch waren. Daarom hebben we besloten om nieuwe visualisaties met wereldkaarten te maken. Tijdens de feedbacksessie hebben we dit nieuwe idee voorgelegd aan de TA, en we hebben besloten dit uit te voeren. Later hebben wij ook een e-mail ontvangen met alle feedbackpunten op een rijtje. Deze hebben wij tijdens het werken aan het project steeds bijgehouden en afgestreept wat we gedaan hadden.

**Zelf reflectie**

Een belangrijk leermoment voor ons was de toepassing van de Pearson correlatiecoëfficiënt, zoals besproken tijdens het college. Deze correlatie hielp ons om de relatie tussen verschillende variabelen in onze dataset beter te begrijpen, met name in de scatterplots die we hebben gemaakt. Bijvoorbeeld, in onze analyse van persvrijheid versus conflicten, identificeerden we een matige correlatie die eerder niet expliciet was benoemd. Door deze inzichten toe te passen, konden we onze conclusies en aanbevelingen versterken.

Tot slot hebben we geleerd dat het belangrijk is om kritisch te blijven over onze datavisualisaties en analyses, vooral in termen van duidelijkheid, interpretatie van indexen en de keuze van correlatiemethoden. Deze kritische benadering heeft ons geholpen om onze data story te versterken en een beter inzicht te bieden in de complexe dynamiek tussen persvrijheid, conflicten en bestuursvormen wereldwijd.


## Werkverdeling

Communicatie over het project vond plaats tijdens de werkcolleges, via berichten of door te bellen. Hier hebben wij afgesproken wie wat doet en konden wij overleggen over onduidelijkheden. Het onderwerp hebben wij gezamenlijk uitgekozen en het zoeken en verwerken van datasets voor de proposal hebben wij samen gedaan.

**Anne** -
Anne heeft gewerkt aan het maken van visualisaties. De grafieken rondom persvrijheid en democratie komen van haar. Ook heeft zij samen met Noor gewerkt aan de wereldkaarten. De code laten aansluiten op de gemaakte dataset is ook door Anne gedaan. Verder was zij ook aanwezig bij de presentatie.

**Glenn** -
Het verwoorden van de argumenten en perspectieven is door Glenn gedaan. Ook heeft hij de grafieken omschreven en ervoor gezorgd dat alle stukken tekst op elkaar aansluiten. Glenn was ook aanwezig bij de presentatie.

**Job** -
Het schoonmaken en samenvoegen van de datasets is door Job gedaan. Ook heeft hij de omschrijving van de variabelen geschreven. Verder heeft hij ervoor gezorgd dat het Jupyter Book op orde was.

**Noor** -
Noor heeft gewerkt aan de visualisaties rondom persvrijheid en conflicten. Ook heeft zij samen met Anne aan de wereldkaarten gewerkt. Ook was zij aanwezig bij de presentatie.


## Referenties

1. ACLED. (2024, 13 februari). <em>ACLED Conflict Index - ACLED.</em> https://acleddata.com/conflict-index/#downloads 
2. <em>Democracy index.</em> (2022, 10 februari). Our World in Data. https://ourworldindata.org/grapher/democracy-index-eiu?tab=table&time=earliest..2023 
3. <em>Index.</em> (z.d.). RSF. https://rsf.org/en/index 
4. Ministerie van Justitie en Veiligheid. (2022, July 6). <em>Vrijheid van meningsuiting.</em> Thema’s | College Voor De Rechten Van De Mens. https://www.mensenrechten.nl/themas/vrijheid-van-meningsuiting 

## Externe links

 - [GitHub repository](https://github.com/J08K/InfoViz) 



## Alles wat hier onder staat kan weg denk ik??

## Ter verduidelijking:


-----
- 'Conflict index':

Een hogere rang (bijv. 1) geeft aan dat er in een land veel/grote conflicten zijn
Een lagere rang (bijv. 150) geeft aan dat er in een land weinig/kleine conflicten zijn

--------
- 'Democratie index':

Een hogere rang (bijv. 1) geeft aan dat er in een land een heersende democratie is
Een lagere rang (bijv. 150) geeft aan dat er in een land geen democratie is

---------
- 'Persvrijheid  index':

Een hogere rang (bijv. 1) geeft aan dat de pers in een land veel vrijheid heeft
Een lagere rang (bijv. 150) geeft aan dat de pers in een land weinig vrijheid heeft.

## Wereldkaart. Glennnnnnnn kijk hier naar :)

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft democratie te laten zien per land (2019-2023)

# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],          # Donker rood
    [34/(32+35), '#FF7377'], # Licht rood
    [35/(32+35), '#0000ff'], # Blauw voor 0
    [36/(32+35), '#90ee90'], # Licht groen
    [1, '#006400']           # Donker groen
]
 
# Maak de choropletenkaart
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['democratie_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in democratie rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.92,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['democratie_verandering'].min(), 0, eind_df['democratie_verandering'].max()],
        ticktext=['Autocratisch', 'Geen verandering', 'Democratisch']
    ),
    zmin=eind_df['democratie_verandering'].min(),
    zmax=eind_df['democratie_verandering'].max(),
)
 
# Maak de figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Democratie Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 
fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

**Pie chart van democratie verandering**

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft conflict te laten zien per land (2019-2023)


# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],            # Donker rood
    [112/(48+113), '#FF7377'], # Licht rood
    [113/(48+113), '#0000ff'], # Blauw
    [114/(48+113), '#90ee90'], # Licht groen
    [1, '#006400']             # Donker groen
]
 
# Maak de choropletenkaart
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['conflict_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in conflict rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.89,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['conflict_verandering'].min(), 0, eind_df['conflict_verandering'].max()],
        ticktext=['Meer conflicten', 'Geen verandering', 'Minder conflicten']
    ),
    zmin=eind_df['conflict_verandering'].min(),
    zmax=eind_df['conflict_verandering'].max(),
)
 
# Maak de figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Conflict Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 
fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

print('Figuur 3')

In [ ]:
# Creëer een wereldkaart om de verandering van rang wat betreft persvrijheid te laten zien per land (2019-2023)

# Maak een kleurenschaal
colorscale = [
    [0, '#9b0000'],          # Donker rood
    [54/(74+55), '#FF7377'], # Licht rood
    [55/(74+55), '#0000ff'], # Blauw
    [56/(74+55), '#90ee90'], # Licht groen
    [1, '#006400']           # Donker groen
]
 
# Maak de choropletenkaart 
trace = go.Choropleth(
    locations=eind_df['land'],
    locationmode='country names',
    z=eind_df['persvrijheid_verandering'],
    colorscale=colorscale,
    hovertemplate='<b>%{location}</b><br>Verandering in persvrijheid rang: %{z}<extra></extra>',
    colorbar=go.choropleth.ColorBar(
        x=0.5,
        y=0.89,
        xanchor='center',
        yanchor='bottom',
        orientation='h',
        len=0.5,
        thickness=10,
        tickmode='array',
        title='',
        tickvals=[eind_df['persvrijheid_verandering'].min(), 0, eind_df['persvrijheid_verandering'].max()],
        ticktext=['Meer persvrijheid', 'Geen verandering', 'Minder persvrijheid']
    ),
    zmin=eind_df['persvrijheid_verandering'].min(),
    zmax=eind_df['persvrijheid_verandering'].max(),
)
 
# Maak de figuur
fig = go.Figure(
    data=[trace],
    layout=go.Layout(
        width=790,
        height=640,
        title='Verandering in Persvrijheid Rang (2019-2023)',
        geo={'showocean': True, 'oceancolor': '#a8d5f2', 'landcolor': '#ffffff'},
        margin={'t': 80, 'r': 20, 'b': 80, 'l': 20},
    )
)
 

fig.update_geos(showcountries=True, showcoastlines=False)
 
fig.show()

print('Figuur 4')

In [ ]:
# Creëer een pie chart van de verandering van de conflict rangen

status_counts = pd.Series(['Verslechterd' if x < 0 else 'Verbeterd' if x > 0 else 'Gelijk gebleven' for x in eind_df['conflict_verandering']]).value_counts()


fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['red', 'green', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in conflict rangschikking sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()

In [ ]:
# Creëer een pie chart van de verandering van de persvrijheid rangen

status_counts = pd.Series(['Verslechterd' if x < 0 else 'Verbeterd' if x > 0 else 'Gelijk gebleven' for x in eind_df['persvrijheid_verandering']]).value_counts()


fig_status = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    marker=dict(colors=['red', 'green', 'blue'])
)])

fig_status.update_layout(
    title='Verandering in Persvrijheid rangschikking sinds 2019 per 2023',
    legend_title='Status'
)

fig_status.show()